In [ ]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import os
import sys
import glob
from scipy.ndimage.interpolation import shift
from scipy.signal import find_peaks
from scipy.signal import find_peaks_cwt
from scipy.signal import peak_prominences
from scipy.signal import peak_widths
import itertools
from scipy.optimize import curve_fit
import csv


#np.set_printoptions(threshold=sys.maxsize)


#define a functioon that saves the fits
def write_hdulist_to(hdulist, fileobj, overwrite=True, **kwargs):
    hdulist.writeto(fileobj, overwrite=True, **kwargs)


''' NIRES header keywords of interest
Dither pattern name: DPATNAME
Dither pattern step size in X: DPATSTPX
Dither pattern step size in Y: DPATSTPY
Current frame dither position X offset: XOFFSET
Current frame dither position Y offset: YOFFSET
Pixel scale: PSCALE
ITIME: Integration time
DATAFILE: image saved data file name
OBJECT: object name
TARGNAME: target name
OBSTYPE: observation type
'''

def plot(data):
    plt.figure(figsize=(16, 8))
    plt.imshow(data,
               origin='lower',
               interpolation='nearest',
               cmap='Greys',
               norm=LogNorm(),
               aspect='auto')
    plt.interactive(True)
    plt.colorbar()

def plot_slices(slices,n):
    x = 2048
    plt.plot(range(x),list(itertools.repeat(slices[n][0],x)),c = 'red')
    plt.plot(range(x),list(itertools.repeat(slices[n][1],x)),c = 'red')
    return

In [ ]:
# read the master flat frame for retrace.
date = '20180630' 
flat_data = fits.getdata('NIRES/'+date+'/m_flat_'+date[2:8]+'_'+'.fits')
# shift the master flat frame.
shift_pix = 3
flat_data_shift = shift(flat_data,shift_pix)
# take the abs of the difference between the shifted flat frame and the origional one.
edges = abs(flat_data-flat_data_shift)

# slice the flat frame horizontally.
# every pair of the slice range covers two edges of upper and lower spectrum.
# except for the first and last pair that only covers one edge.

# the first and last slice range contains one edge, while the rest ranges contains two edges
# of the upper edge of the lower spectrum and the lower edge of the upper spectrum.
slices_1 = [[151,251],[251,395],[390,523]]
slices_2 = [[590,668],[767,832],[937,955]]
slices = slices_1+slices_2

def format(value):
    return "%.3f" % value
with open('slices.txt', 'w') as f:
    formatted = [[format(v) for v in r] for r in slices]
    f.write(str(formatted))

In [ ]:
plt.figure(figsize=(16, 8))
plt.imshow(edges,
           origin='lower',
           interpolation='nearest',
           cmap='Greys',
           norm = LogNorm(),
           aspect='auto')
plt.interactive(True)
plt.colorbar()

for n in range(len(slices)):
    plot_slices(slices,n)

In [ ]:
n = 50
i = 1

lo = slices[i][0]
up = slices[i][1]

x = range(len(edges[lo:up,500]))
y = (edges[lo:up,n-1]+edges[lo:up,n]+edges[lo:up,n+1])/3

plt.figure(figsize=(20, 12))
plt.plot(x, y)

peaks,properties = find_peaks(y,width=(2,4.5),prominence=0.04)

plt.plot(peaks,y[peaks],'x')

prom = peak_prominences(y,peaks)[0]
width = peak_widths(y,peaks)
central = (properties['right_ips']-properties['left_ips'])/2+properties['left_ips']+lo-(shift_pix/2)

plt.hlines(*width[1:], color="C3")
print(peaks)
print(prom)
print(width)
print(properties)
print(central)

In [ ]:
def find_edge_coord(slices, i):
    edge_coord1 = []
    edge_coord2 = []
    x_lim = []
    # define y coordinates of the slice range.
    lo = slices[i][0]
    up = slices[i][1]
    # go through the y axis, find the edges by finding the peaks along the y-axis
    # average over every 3 pixels on the x-aixs.
    for x in range(2, len(flat_data[1]), 3):
        # if x is the last element, pass.
        if x == 2048: pass
        else:
            # average over every 3 pixels along x-axis.
            edge = (edges[lo:up, x - 1] + edges[lo:up, x] +
                    edges[lo:up, x + 1]) / 3
            # find the peaks and prominences along y-axis.
            peaks, properties = find_peaks(edge, width=(2, 4), prominence=0.04)
            prominences = properties['prominences']
            # define the y-coordinate of a peak to be the middle value of that peak's FWHM.
            peak_coord = (properties['right_ips'] +
                          properties['left_ips']) / 2 + lo - (shift_pix / 2)
            ########################################################################
            ########################################################################
            # for the first and last slice range there's only one edge to be found.
            if i == 0:
                # provide a limit on the x-axis for the spectrum
                # to avoid the over scanned area.
                edge_coord2.append(np.nan)
                x_lim.append(1100)
                if x <= 1100:
                    # if no peak is found, append zero value to the list.
                    if peak_coord.size == 0:
                        edge_coord1.append(0)
                    elif prominences.size >= 1:
                        # the peaks with the max values.
                        peak_id1 = np.argmax(prominences)
                        edge_coord1.append(peak_coord[peak_id1])
                else:
                    pass
                    ########################################################################
            ########################################################################
            # for the first and last slice range there's only one edge to be found.
            if i == 5:
                x_lim.append(2048)
                edge_coord2.append(np.nan)
                # if no peak is found, append zero value to the list.
                if peak_coord.size == 0:
                    edge_coord1.append(0)
                elif prominences.size >= 1:
                    # find the peaks with the max values.
                    peak_id1 = np.argmax(prominences)
                    edge_coord1.append(peak_coord[peak_id1])
            ########################################################################
            ########################################################################
            # for the second slice range, there are two edeges but the lower edge
            # ends before the upper edge.
            elif i == 1:
                ########################################################################
                # provide a limit on the x-axis for the spectrum
                # to avoid the over scanned area.
                x_lim.append(1000)
                if x <=1000:
                    # if found less than two peaks, append zero value to the list.
                    if peak_coord.size <= 1:
                        edge_coord1.append(0)
                        edge_coord2.append(0)
                    # if found exact two peaks
                    elif prominences.size == 2:
                        # find the max and 2nd max peak values
                        # for the first slice range peaks of the upper edge are always greater
                        peak_id1 = np.argmax(prominences)
                        peak_id2 = np.where(prominences == np.partition(
                            prominences, -2)[-2])[0][0]
                        edge_coord1.append(peak_coord[peak_id1])
                        edge_coord2.append(peak_coord[peak_id2])

                    elif prominences.size > 2:
                        peak_id1 = np.argmax(prominences)
                        peak_id2 = np.where(prominences == np.partition(
                            prominences, -2)[-2])[0][0]
                        peak_id3 = np.where(prominences == np.partition(
                            prominences, -3)[-3])[0][0]

                        if peak_coord[peak_id1] > peak_coord[
                                peak_id2] and peak_coord[
                                    peak_id1] < peak_coord[peak_id3]:
                            edge_coord1.append(peak_coord[peak_id1])
                            edge_coord2.append(peak_coord[peak_id2])
                        elif peak_coord[peak_id1] < peak_coord[
                                peak_id2] and peak_coord[
                                    peak_id1] > peak_coord[peak_id3]:
                            edge_coord1.append(peak_coord[peak_id1])
                            edge_coord2.append(peak_coord[peak_id3])

                        elif peak_coord[peak_id1] > peak_coord[
                                peak_id2] and peak_coord[
                                    peak_id1] > peak_coord[peak_id3]:
                            edge_coord1.append(peak_coord[peak_id1])
                            edge_coord2.append(peak_coord[peak_id2])

                        elif peak_coord[peak_id1] < peak_coord[
                                peak_id2] and peak_coord[
                                    peak_id1] < peak_coord[peak_id3]:
                            peak_id4 = np.where(prominences == np.partition(
                                prominences, -4)[-4])[0][0]
                            edge_coord1.append(peak_coord[peak_id1])
                            edge_coord2.append(peak_coord[peak_id4])
                ########################################################################
                elif x>1000:
                    if peak_coord.size == 0:
                        edge_coord1.append(0)
                    elif prominences.size >= 1:
                        # the peak with the max value
                        peak_id1 = np.argmax(prominences)
                        edge_coord1.append(peak_coord[peak_id1])
            ########################################################################
            ########################################################################
            # for all the other slice ranges
            elif i == 2 or i == 3 or i == 4:
                x_lim.append(2048)
                if x<=2048:
                    if peak_coord.size <= 1:
                        edge_coord1.append(0)
                    elif prominences.size > 1:
                        # find the max and 2nd max peaks
                        peak_id1 = np.argmax(prominences)
                        peak_id2 = np.where(
                            prominences == np.partition(prominences, -2)[-2])[0][0]
                        # determine which edge the peak belongs two by comparing
                        # their coordinates.
                        if peak_coord[peak_id1] > peak_coord[peak_id2]:
                            edge_coord1.append(peak_coord[peak_id1])
                            edge_coord2.append(peak_coord[peak_id2])
                        else:
                            edge_coord1.append(peak_coord[peak_id2])
                            edge_coord2.append(peak_coord[peak_id1])
    return edge_coord1, edge_coord2, x_lim[0]

In [ ]:
def eliminate_outliers(edge_coord1, out):
    # eliminate the outliers by
    # giving them zero values
    for i in range(len(edge_coord1)):
        local_median_elements = []
        # the first 100 elements
        if i <= 42:
            # define the elements equal or greater than the 'out' parameter
            # comparing with the local median of 40 values
            for n in range(80):
                local_median_elements.append(edge_coord1[i + n])
            if abs(edge_coord1[i] - np.median(local_median_elements)) >= out:
                edge_coord1[i] = 0
        # the last 100 elements
        elif i >= len(edge_coord1) - 41:
            for n in range(80):
                local_median_elements.append(edge_coord1[i - n])
            if abs(edge_coord1[i] - np.median(local_median_elements)) >= out:
                edge_coord1[i] = 0
        # everything else
        else:
            for n in range(40):
                local_median_elements.append(edge_coord1[i - n])
                local_median_elements.append(edge_coord1[i + n])
            if abs(edge_coord1[i] - np.median(local_median_elements)) >= out:
                edge_coord1[i] = 0
    return edge_coord1

In [ ]:
def replace_zeros(edge_coord1, replace_range):
    # replace the zero values with the mean of replace_range(number) pxiel values
    for index, value in enumerate(edge_coord1):
        if value == 0:
            replace = []
            # for the last elements
            if index + replace_range >= len(edge_coord1) - 1:
                # determine if the designated replacement is zero or not
                # if not zero then write to the replace list
                for r in range(replace_range):
                    if not edge_coord1[index - r] == 0:
                        replace.append(edge_coord1[index - r])
                # calculate the mean of the nonzero values in the replace list
                replace = np.mean(replace)
                # if no mean value is avaliable, remain the origional value
                if np.isnan(replace): edge_coord1[index] = value
                # else replace the zero value with the replacement value
                else:
                    edge_coord1[index] = value + replace
            # for the first elements
            elif index <= replace_range:
                for r in range(replace_range):
                    if not edge_coord1[index + r] == 0:
                        replace.append(edge_coord1[index + r])
                replace = np.mean(replace)
                if np.isnan(replace): edge_coord1[index] = value
                else:
                    edge_coord1[index] = value + replace
            # for everything else
            else:
                for r in range(replace_range):
                    if not edge_coord1[index -
                                       r] == 0 and not edge_coord1[index +
                                                                   r] == 0:
                        replace.append(edge_coord1[index - r])
                        replace.append(edge_coord1[index + r])
                replace = np.mean(replace)
                if np.isnan(replace): edge_coord1[index] = value
                else:
                    edge_coord1[index] = value + replace

        else:
            pass

    return edge_coord1

#### Slice 0: out=15, replace_range=20
#### Slice 1: out=12, replace_range=30
#### Slice 2:
#### Slice 3:
#### Slice 4:
#### Slice 5:

In [ ]:
#out range
n = 4
out = 4
replace_range = 30
edge_coord,edge_coord2,x_lim = find_edge_coord(slices,n)

edge_coord = eliminate_outliers(edge_coord,out)
edge_coord = replace_zeros(edge_coord,replace_range)
edge_coord = replace_zeros(edge_coord,replace_range)

x = np.linspace(2,x_lim,len(edge_coord))
polyfit_parameters = np.polyfit(x,edge_coord,15)
pn = np.poly1d(polyfit_parameters)

plt.figure(figsize=(12,8))
plt.scatter(x,edge_coord,marker='.')
plt.plot(x,pn(x),c='red')

formatted_polyfit_parameters =  [format(r)  for r in polyfit_parameters]
print(x_lim)
print(''formatted_polyfit_parameters)

edge_coord2 = eliminate_outliers(edge_coord2,out)
edge_coord2 = replace_zeros(edge_coord2,replace_range)
edge_coord2 = eliminate_outliers(edge_coord2,out)
edge_coord2 = replace_zeros(edge_coord2,replace_range)

x2 = np.linspace(2,x_lim,len(edge_coord2))
polyfit_parameters = np.polyfit(x2,edge_coord2,15)
pn2 = np.poly1d(polyfit_parameters)

plt.figure(figsize=(12,8))
plt.scatter(x2,edge_coord2,marker='.')
plt.plot(x2,pn2(x2),c='red')

formatted_polyfit_parameters2 =  [format(r)  for r in polyfit_parameters]
print(formatted_polyfit_parameters2)

In [ ]:
plt.figure(figsize=(16, 8))
plt.imshow(flat_data,
           origin='lower',
           interpolation='nearest',
           cmap='Greys',
           norm = LogNorm(),
           aspect='auto')
plt.interactive(True)
plt.colorbar()
plt.plot(x,pn(x),c='red')
plt.plot(x2,pn2(x2),c='blue')